## Load sampled images and download detailed metadata

In [28]:
import sys
sys.path.append("D:\\projects\\geo_image_experiments\\src")
import importlib
import mapillary_utils as mu
importlib.reload(mu)

<module 'mapillary_utils' from 'D:\\projects\\geo_image_experiments\\src\\mapillary_utils.py'>

In [29]:
import requests
from dotenv import load_dotenv
from os import getenv

from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np

import sqlite3
import json

import requests


pd.set_option('display.max_colwidth', None)

In [30]:
test_dir = "d://projects_working_directories//202408_pano_images"
file = "360_images_2024-09-08.db"

images_dir = f"{test_dir}//images"

db_file = f"{test_dir}//{file}"

load_dotenv()
API_KEY = getenv("MAPILLARY_CLIENT_TOKEN")

In [31]:
# load sampled images 
conn = sqlite3.connect(f"{test_dir}//{file}")
df_sampled_images = pd.read_sql('select * from sampled_images', conn)
conn.close()

In [32]:
df_sampled_images.sample()

,image_lat,image_lon,residual,image_id,camera_type,is_pano,camera_focal_len,camera_k1,camera_k2,image_path,error,image_url,altitude,compass_angle,exif_orientation,make,model,country,city
63,-6.348579,106.783767,53.320903,1739890546463224,spherical,1,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An_bJ8Z4IhfW90H8zTHA3VkDFoSLjY7jUpLMegpgY0IGNbnIr8BIKBVDW7hMHpctpZZSbU1sC8D5ceX9_nnNGn3EpeV_fAc7FVIsic8yaDb8YYtqFSPsiXsUYCTtq4nIGD_SbYbRyykE5N1UItlBPQ?ccb=10-5&oh=00_AYC0sST-LWqjzqiKPP2km357fp3G3NLrKq9-U6BLnTVCYw&oe=6705206A&_nc_sid=201bca,85.934,355.54263016766,1,GoPro,GoPro Max,Indonesia,Jakarta


In [6]:
#image_id = '856814222158020'

#image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator='thumb_2048_url', image_dir=images_dir)
#print("Metadata:", metadata.keys)

In [12]:
image_size_cycle = [
    #'thumb_original_url','thumb_2048_url'
    'thumb_2048_url'
]

for image_id in df_sampled_images['image_id'].tolist():
    for image_size in image_size_cycle:
        image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator=image_size, image_dir=images_dir)
        try:
            df_temp = pd.DataFrame.from_dict([metadata])
            df_metadata = pd.concat([df_metadata,df_temp], ignore_index=True)
        except:
            df_metadata = pd.DataFrame.from_dict([metadata])

{'type': 'Point', 'coordinates': [106.92201489997, -6.2392912999722]}
{'type': 'Point', 'coordinates': [106.92393269997, -6.2388519]}
{'type': 'Point', 'coordinates': [106.92097699997, -6.2415329]}
{'type': 'Point', 'coordinates': [106.92284119997, -6.2381000999722]}
{'type': 'Point', 'coordinates': [106.9219152, -6.2381371]}
{'type': 'Point', 'coordinates': [106.9238595, -6.2388244999722]}
{'type': 'Point', 'coordinates': [106.9242705, -6.2388643]}
{'type': 'Point', 'coordinates': [106.92368729997, -6.2405701]}
{'type': 'Point', 'coordinates': [106.9222079, -6.2388754999722]}
{'type': 'Point', 'coordinates': [106.9209847, -6.241628]}
{'type': 'Point', 'coordinates': [106.9222349, -6.2388563]}
{'type': 'Point', 'coordinates': [106.92296819997, -6.2402395999722]}
{'type': 'Point', 'coordinates': [106.92225929997, -6.2382020999722]}
{'type': 'Point', 'coordinates': [106.92219769997, -6.2389337999722]}
{'type': 'Point', 'coordinates': [106.92358869997, -6.2386032]}
{'type': 'Point', 'coor

In [13]:
def is_image_original_size(row):
    if (row.original_height == row.height) & (row.original_width == row.width):
        return 1
    else:
        return 0

df_metadata['is_image_original_size'] = df_metadata.apply(is_image_original_size, axis=1)

In [14]:
conn = sqlite3.connect(db_file)

df_metadata.to_sql('image_metadata', con=conn, if_exists='replace', index=False)



# Close the connection
conn.close()

## Add Exif Data Back

In [18]:
from PIL import Image
import piexif

In [15]:
df_metadata = df_metadata.merge(df_sampled_images[['altitude','make','model','image_id']], how='left', left_on='image_id', right_on='image_id')

In [17]:
df_metadata.sample(10)

,guid,image_source,image_id,captured_at_unix,lat,lon,original_height,original_width,height,width,...,compass_angle,is_pano,camera_focal_len,camera_k1,camera_k2,image_path_on_disk,is_image_original_size,altitude,make,model
6,e417ec130c1a4566af1009a9faf9d153,mapillary,380911181018609,1696839228000,-6.238100,106.922841,2880,5760,1024,2048,...,6.378293,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_380911181018609_thumb_2048_url.jpg,0,29.143,GoPro,GoPro Max
85,6c61858d81b142b9a846f577b1570829,mapillary,172474141941061,1658411862000,-7.198741,112.734207,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_172474141941061_thumb_2048_url.jpg,0,31.725,GoPro,GoPro Max
90,0dc875c48fbd42c8ba1c99d4ea1b0cfe,mapillary,456417362984425,1658413211000,-7.200475,112.732933,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_456417362984425_thumb_2048_url.jpg,0,31.787,GoPro,GoPro Max
42,8f36d7079e76485ca5459142da411cab,mapillary,621262016879258,1692518295000,-6.211813,106.744359,2880,5760,1024,2048,...,153.134434,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_621262016879258_thumb_2048_url.jpg,0,16.352,GoPro,GoPro Max
65,4dfe9e0526e0449e90fccdc8f0b87ab3,mapillary,2379488058858289,1658589178000,-7.240682,112.608088,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_2379488058858289_thumb_2048_url.jpg,0,42.141,GoPro,GoPro Max
110,e868aec314aa41058a198eeb41db4e8f,mapillary,2133236863523552,1658246534000,-7.339105,112.757954,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_2133236863523552_thumb_2048_url.jpg,0,31.491,GoPro,GoPro Max
61,e6be5a8c937f4d1cab80646f888b5aa8,mapillary,539032611221664,1653568985000,-6.369993,106.984811,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_539032611221664_thumb_2048_url.jpg,0,77.268,GoPro,GoPro Max
14,fdb96c3b0db74f2fa03995b87263a7b7,mapillary,6939473759422246,1696839072000,-6.240240,106.922968,2880,5760,1024,2048,...,190.531324,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_6939473759422246_thumb_2048_url.jpg,0,37.104,GoPro,GoPro Max
53,b7bdf3044dc044a0b0a31b7982abebc4,mapillary,565807408235147,1653568933000,-6.371727,106.984358,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_565807408235147_thumb_2048_url.jpg,0,77.332,GoPro,GoPro Max
26,62be148382c44f0187e65fac768c7126,mapillary,1215236129641347,1692492987000,-6.210902,106.743816,2880,5760,1024,2048,...,338.139000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1215236129641347_thumb_2048_url.jpg,0,31.882,GoPro,GoPro Max


In [19]:
from geopy import Point
from geopy.distance import distance

# Decimal to DMS
def decimal_to_dms(lat, lon):
    point = Point(latitude=lat, longitude=lon)
    
    
    lat_dms = point.format().split(',')[0]
    lon_dms = point.format().split(',')[1]
    
    return lat_dms, lon_dms


lat, lon = -6.222932, 106.833616
lat_dms, lon_dms = decimal_to_dms(lat, lon)
print(f"Latitude: {lat_dms}, Longitude: {lon_dms}")

Latitude: 6 13m 22.5552s S, Longitude:  106 50m 1.0176s E


In [20]:


from datetime import datetime

def unix_to_exif_datetime(unix_timestamp):
    # Convert Unix timestamp to a datetime object
    dt = datetime.utcfromtimestamp(unix_timestamp)
    
    # Format datetime as "YYYY:MM:DD HH:MM:SS"
    exif_datetime = dt.strftime("%Y:%m:%d %H:%M:%S")
    
    return exif_datetime
    
unix_to_exif_datetime(1000 / 1000)


'1970:01:01 00:00:01'

In [21]:
def build_exif_0th(make='', model=''):
    
    zero_th =  {
        piexif.ImageIFD.Make: make,
        piexif.ImageIFD.Model: model
    }

    return zero_th

def build_exif_exif(datetime=1000, lensmake=None, lensmodel=None):
    
    
    exif = {
        piexif.ExifIFD.DateTimeOriginal: unix_to_exif_datetime(datetime / 1000),
        piexif.ExifIFD.LensMake: "",
        piexif.ExifIFD.LensModel: ""
    }

    return exif

#print(build_exif_0th(make='GoPro', model='360 Max'))
#print(build_exif_exif(datetime=1000))


In [22]:
def build_exif_gps(lat, lon):
    # pass in lat/lon in decimal format
    # then convert it to a string
    # then convert to exif format
    # 
    # can probably convert directly but
    # I don't know how
    

    
    def split_elements(element):
        def to_rational(number):
            # Convert a number to a rational tuple (numerator, denominator)
            return (int(number * 1000), 1000)
    
    
        degrees, minutes, seconds, reference = element.split()
        degrees = abs(int(degrees))
        minutes = int(minutes[0:-1])
        seconds = to_rational(float(seconds[0:-1]))

        return ((degrees,1),(minutes,1),seconds) , reference

    
    point = Point(lat, lon)
    string_point = point.format()
    string_lat, string_lon = string_point.split(', ')
    
    print(string_point)
    
    lat_elements, lat_ref = split_elements(string_lat)
    lon_elements, lon_ref = split_elements(string_lon)

    gps_ifd = {
        piexif.GPSIFD.GPSLatitudeRef: lat_ref.encode(),
        piexif.GPSIFD.GPSLatitude: lat_elements,
        piexif.GPSIFD.GPSLongitudeRef: lon_ref.encode(),
        piexif.GPSIFD.GPSLongitude: lon_elements,
    }

    return gps_ifd

build_exif_gps(37.7749, -122.4194)

37 46m 29.64s N, 122 25m 9.84s W


{1: b'N',
 2: ((37, 1), (46, 1), (29640, 1000)),
 3: b'W',
 4: ((122, 1), (25, 1), (9840, 1000))}

In [23]:
def generate_exif_data(row):
    
    zero_th = build_exif_0th(make=row.make, model=row.model)
    exif = build_exif_exif(datetime=row.captured_at_unix)
    gps = build_exif_gps(row.lat, row.lon)

    print(zero_th)

    exif_data = {
        "0th" : zero_th,
        "Exif" : exif,
        "GPS" : gps

    }

    return exif_data

df_metadata['exif_dict'] = df_metadata.apply(generate_exif_data, axis=1)

6 15m 48.074s S, 106 47m 3.80544s E
{271: nan, 272: nan}
7 25m 35.8144s S, 112 45m 0.38808s E
{271: nan, 272: nan}
7 16m 23.6046s S, 112 38m 18.8034s E
{271: nan, 272: nan}
6 14m 21.4487s S, 106 55m 19.2536s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 19.8668s S, 106 55m 26.1577s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 29.5184s S, 106 55m 15.5172s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 17.1604s S, 106 55m 22.2283s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 17.2936s S, 106 55m 18.8947s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 19.7682s S, 106 55m 25.8942s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 19.9115s S, 106 55m 27.3738s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 26.0524s S, 106 55m 25.2743s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 19.9518s S, 106 55m 19.9484s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 29.8608s S, 106 55m 15.5449s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 19.8827s S, 106 55m 20.0456s E
{271: 'GoPro', 272: 'GoPro Max'}
6 14m 24.8626s S, 106 55m 22.6855s E
{271: 'GoPro', 272: 

In [23]:
df_metadata = df_metadata.merge(df_sampled_images, how='left', left_on='image_id', right_on='image_id')
df_metadata.to_excel("c://temp//foo.xlsx")

In [24]:
def add_exif(image_path, exif_data):

    # Open the image
    image = Image.open(image_path)

    output_path = image_path + "with_exif.jpg"

    # Extract existing EXIF data (if any)
    existing_exif = image.info.get('exif')


    # Load existing EXIF data
    exif_dict = piexif.load(existing_exif) if existing_exif else {"0th": {}, "Exif": {}, "GPS": {}, "1st": {}, "thumbnail": None}
    print(exif_dict)

    # Modify or add new EXIF tags
    for ifd in ("0th", "Exif", "GPS", "1st"):
        if ifd in exif_data:
            for tag, value in exif_data[ifd].items():
                print(f"{tag} - {value} - {ifd}")
                exif_dict[ifd][tag] = value

    # Convert the modified EXIF data back to binary
    exif_bytes = piexif.dump(exif_dict)

    # Save the image with the new EXIF data
    print(output_path)
    image.save(output_path, exif=exif_bytes)


exif_data = {
    "0th": {
        piexif.ImageIFD.Make: "Camera Brand",
        piexif.ImageIFD.Model: "Camera Model",
        piexif.ImageIFD.Software: "My Software"
    },
    "Exif": {
        piexif.ExifIFD.DateTimeOriginal: "2024:08:31 12:34:56",
        piexif.ExifIFD.LensMake: "Lens Brand",
        piexif.ExifIFD.LensModel: "Lens Model"
    },
    "GPS": {
        piexif.GPSIFD.GPSLatitudeRef: "N",
        piexif.GPSIFD.GPSLatitude: ((40, 1), (0, 1), (0, 100)),  # 40 degrees
        piexif.GPSIFD.GPSLongitudeRef: "W",
        piexif.GPSIFD.GPSLongitude: ((74, 1), (0, 1), (0, 100)),  # 74 degrees
    }
}

#add_exif('d://projects_working_directories//202408_pano_images//images//mapillary_1332341057452601_thumb_2048_url.jpg', exif_data)

In [25]:
df_metadata.apply(lambda x: add_exif(x.image_path_on_disk, x.exif_dict), axis=1)

{'0th': {}, 'Exif': {}, 'GPS': {}, '1st': {}, 'thumbnail': None}
271 - nan - 0th
272 - nan - 0th
36867 - 2023:09:30 11:59:43 - Exif
42035 -  - Exif
42036 -  - Exif
1 - b'S' - GPS
2 - ((6, 1), (15, 1), (48074, 1000)) - GPS
3 - b'E' - GPS
4 - ((106, 1), (47, 1), (3805, 1000)) - GPS


ValueError: "dump" got wrong type of exif value.
271 in 0th IFD. Got as <class 'float'>.

In [26]:
df_metadata

,guid,image_source,image_id,captured_at_unix,lat,lon,original_height,original_width,height,width,...,is_pano,camera_focal_len,camera_k1,camera_k2,image_path_on_disk,is_image_original_size,altitude,make,model,exif_dict
0,e2190d1285534663b87044c0471cc5dc,mapillary,1505018570392136,1696075183000,-6.263354,106.784390,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1505018570392136_thumb_2048_url.jpg,0,NaN,NaN,NaN,"{'0th': {271: nan, 272: nan}, 'Exif': {36867: '2023:09:30 11:59:43', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((6, 1), (15, 1), (48074, 1000)), 3: b'E', 4: ((106, 1), (47, 1), (3805, 1000))}}"
1,a01f74ea1fa04e06846a28cf3f15a3db,mapillary,1095545924696245,1658328383000,-7.426615,112.750108,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1095545924696245_thumb_2048_url.jpg,0,NaN,NaN,NaN,"{'0th': {271: nan, 272: nan}, 'Exif': {36867: '2022:07:20 14:46:23', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((7, 1), (25, 1), (35814, 1000)), 3: b'E', 4: ((112, 1), (45, 1), (388, 1000))}}"
2,f3c433be37a345c0ac8d333b78b859cb,mapillary,1564726257278323,1658593288000,-7.273224,112.638557,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1564726257278323_thumb_2048_url.jpg,0,NaN,NaN,NaN,"{'0th': {271: nan, 272: nan}, 'Exif': {36867: '2022:07:23 16:21:28', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((7, 1), (16, 1), (23604, 1000)), 3: b'E', 4: ((112, 1), (38, 1), (18803, 1000))}}"
3,29f6d6c45feb4ff587b39054a8ebd7c2,mapillary,376567711926026,1696780830000,-6.239291,106.922015,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_376567711926026_thumb_2048_url.jpg,0,35.195,GoPro,GoPro Max,"{'0th': {271: 'GoPro', 272: 'GoPro Max'}, 'Exif': {36867: '2023:10:08 16:00:30', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((6, 1), (14, 1), (21448, 1000)), 3: b'E', 4: ((106, 1), (55, 1), (19253, 1000))}}"
4,815bd099fd174a95bfa4c3f31cd0bc3c,mapillary,1145495376858585,1696838178000,-6.238852,106.923933,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1145495376858585_thumb_2048_url.jpg,0,20.785,GoPro,GoPro Max,"{'0th': {271: 'GoPro', 272: 'GoPro Max'}, 'Exif': {36867: '2023:10:09 07:56:18', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((6, 1), (14, 1), (19866, 1000)), 3: b'E', 4: ((106, 1), (55, 1), (26157, 1000))}}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,e76a74fb34894978b31828d91514c143,mapillary,321026643502982,1658225521000,-7.240029,112.770178,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_321026643502982_thumb_2048_url.jpg,0,37.280,GoPro,GoPro Max,"{'0th': {271: 'GoPro', 272: 'GoPro Max'}, 'Exif': {36867: '2022:07:19 10:12:01', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((7, 1), (14, 1), (24104, 1000)), 3: b'E', 4: ((112, 1), (46, 1), (12641, 1000))}}"
128,072bffb0c6df44eb9e80a5fb795c36f2,mapillary,776924910005065,1658225509000,-7.239161,112.770513,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_776924910005065_thumb_2048_url.jpg,0,37.035,GoPro,GoPro Max,"{'0th': {271: 'GoPro', 272: 'GoPro Max'}, 'Exif': {36867: '2022:07:19 10:11:49', 42035: '', 42036: ''}, 'GPS': {1: b'S', 2: ((7, 1), (14, 1), (20979, 1000)), 3: b'E', 4: ((112, 1), (46, 1), (13846, 1000))}}"
129,a0a57c78f2e24a0e92f9d49558823362,mapillary,1736516810032032,1658396002000,-7.239379,112.770180,2880,5760,1024,2048,...,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1736516810032032_thumb_2048_url.jpg,0,39.054,GoPro,GoPro Max,"{'0th': {271: 'GoPro', 272: 'GoPro Max'}, 'Exif': {36867: '2022:07:21 09:33:22', 42035: '', 42036: ''}, 'GPS':

In [27]:
df_metadata.to_excel("c://temp//metadata.xlsx")